# Mega Bar Plot
at the end of this notebook will be one bar plot that will contain experiments ssp585 and historical from three different models: ESM4, CM4 and CM4x compared to ERA5 data

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cmocean
import cartopy.crs as ccrs
import matplotlib.path as mpath
import nc_time_axis
import pandas as pd
import matplotlib as mpl
import momlevel as ml
import gcsfs
import zarr
from matplotlib.ticker import FormatStrFormatter
import cftime
import warnings

In [11]:
mask_CM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/CM4_mask_antarctica.nc')
mask_ESM4= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ESM4_mask_antarctica.nc')
mask_ERA5= xr.open_dataset('/home/pachamama/shared/antarctica_mask/ERA5_mask_antarctica.nc')
mask_CM4X= xr.open_dataset('/home/pachamama/shared/antarctica_mask/CM4X_mask_antarctica.nc')

In [33]:
def standard_grid_cell_area(lat, lon, rE=6371.0e3):
    """ computes the cell area for a standard spherical grid """
    dLat = lat[1] - lat[0]
    dLon = lon[1] - lon[0]
    area = np.empty((len(lat), len(lon)))
    for j in range(0, len(lat)):
        for i in range(0, len(lon)):
            lon1 = lon[i] + dLon / 2
            lon0 = lon[i] - dLon / 2
            lat1 = lat[j] + dLat / 2
            lat0 = lat[j] - dLat / 2
            area[j, i] = (
                (np.pi / 180.0)
                * rE
                * rE
                * np.abs(np.sin(np.radians(lat0)) - np.sin(np.radians(lat1)))
                * np.abs(lon0 - lon1)
            )
    return area

# CM4x Historical 

In [13]:
cm4x_uas=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/historical/atmos/monthly/u_ref/*.nc')
cm4x_uas

<xarray.Dataset>
Dimensions:     (time: 1980, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    u_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.185001-185412.u_ref.nc
    title:                 CM4_historical_c192_OM4p25
    associated_files:      area: 18540101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Sat Dec 24 23:27:31 2022
    hostname:              pp038
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [14]:
cm4x_vas=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/historical/atmos/monthly/v_ref/*.nc')
cm4x_vas

<xarray.Dataset>
Dimensions:     (time: 1980, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    v_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.185001-185412.v_ref.nc
    title:                 CM4_historical_c192_OM4p25
    associated_files:      area: 18540101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Sat Dec 24 23:27:31 2022
    hostname:              pp038
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [16]:
cm4x_uas_seasonal=cm4x_uas.u_ref.sel(time=slice('1995-01-16','2014-12-16')).groupby('time.season').mean(dim='time')
cm4x_uas_seasonal

<xarray.DataArray 'u_ref' (season: 4, lat: 360, lon: 576)>
dask.array<stack, shape=(4, 360, 576), dtype=float32, chunksize=(1, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    long_name:      zonal wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [18]:
cm4x_vas_seasonal=cm4x_vas.v_ref.sel(time=slice('1995-01-16','2014-12-16')).groupby('time.season').mean(dim='time')
cm4x_vas_seasonal

<xarray.DataArray 'v_ref' (season: 4, lat: 360, lon: 576)>
dask.array<stack, shape=(4, 360, 576), dtype=float32, chunksize=(1, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Attributes:
    long_name:      meridional wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [23]:
cm4x_vas_use=cm4x_vas.v_ref.sel(time=slice('1995-01-16','2014-12-16'))
cm4x_uas_use=cm4x_uas.u_ref.sel(time=slice('1995-01-16','2014-12-16'))

In [28]:
cm4x_his_mergewinds = xr.merge([cm4x_vas_use,cm4x_uas_use]) # merge winds to make it easy
cm4x_his_mergewinds_xmerge = cm4x_his_mergewinds.lon #extracting the lon component fomr the mergewinds array
cm4x_his_mergewinds_ymerge = cm4x_his_mergewinds.lat
cm4x_his_mergewinds_umerge = cm4x_his_mergewinds.u_ref 
cm4x_his_mergewinds_vmerge = cm4x_his_mergewinds.v_ref
cm4x_his_speedmerge = np.sqrt(np.square(cm4x_his_mergewinds_umerge) + np.square(cm4x_his_mergewinds_vmerge)) ## calculating speed using vector formula
## average
cm4x_his_timeaveragemerge = cm4x_his_speedmerge.sel(time=slice('1995-01-16','2014-12-16')).mean(dim='time')

In [29]:
cm4x_his_speedmerge

<xarray.DataArray (time: 240, lat: 360, lon: 576)>
dask.array<sqrt, shape=(240, 360, 576), dtype=float32, chunksize=(60, 360, 576), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * time     (time) object 1995-01-16 12:00:00 ... 2014-12-16 12:00:00

In [30]:
## merging the UAS seasonal and Vas seasonal to get merge seasonal
cm4x_his_seasonalmerge = xr.merge([cm4x_uas_seasonal,cm4x_vas_seasonal])
cm4x_his_seasonalmerge

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 576, season: 4)
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.3125 0.9375 1.562 2.188 ... 357.8 358.4 359.1 359.7
  * season   (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    u_ref    (season, lat, lon) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
    v_ref    (season, lat, lon) float32 dask.array<chunksize=(1, 360, 576), meta=np.ndarray>
Attributes:
    long_name:      zonal wind component at 10 m
    units:          m/s
    valid_range:    [-400.  400.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    interp_method:  conserve_order2

In [31]:
## calculating the seasonal wind speed
cm4x_his_seasonal_speedmerge = np.sqrt(np.square(cm4x_his_seasonalmerge.u_ref) + np.square(cm4x_his_seasonalmerge.v_ref))

## slicing the summer speed
cm4x_his_Summer_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('DJF')).mean(dim='season')
## Slicing summer season from seasonalmerge
cm4x_his_Summer_szn= cm4x_his_seasonalmerge.sel(season='DJF')

## slicing seasonalspeed to get speed for winter and averaging it
cm4x_his_Winter_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('JJA')).mean(dim='season')
cm4x_his_Winter_szn= cm4x_his_seasonalmerge.sel(season='JJA')

## FALL seasonal speed merge and averaging
cm4x_his_Fall_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('MAM')).mean(dim='season')
cm4x_his_Fall_szn= cm4x_his_seasonalmerge.sel(season='MAM')

## slicing seasonalspeed to get spring values and averaging it
cm4x_his_Spring_szn_speed = cm4x_his_seasonal_speedmerge.sel(season=slice('SON')).mean(dim='season')
cm4x_his_Spring_szn= cm4x_his_seasonalmerge.sel(season='SON')

In [34]:
lat_CM4x_historical = cm4x_his_speedmerge.coords['lat'].values #1-D numpy array of model's latitude values
lon_CM4x_historical = cm4x_his_speedmerge.coords['lon'].values #1-D numpy array of model's longitude values
# make sure to replace the name of the data array if you chose a
# different name and use the appropriate coordinate names within the ['']
# based on the coordinates of lat / lon in your dataset.
# Pass the arrays to the standard_grid_cell_area function
areacell = standard_grid_cell_area(lat_CM4x_historical,lon_CM4x_historical)

# The areacella_xarray function outputs a numpy array. Convert this back into
# and xarray DataArray for further use. If this is a field you will be using
# often. You may wish to save this newly created DataArray as a Dataset and save
# as a netcdf file.
# here we assign dimensions and coordinates that are the same as our original SST array:
areacell_CM4x_historical = xr.DataArray(areacell,dims=({'lat':180, 'lon':360}),\
                                                coords=(cm4x_his_speedmerge.coords['lat'],\
                                                        cm4x_his_speedmerge.coords['lon']))

In [35]:
weights_cm4x=areacell_CM4x_historical

In [37]:
weight_ross_CM4x_historical=(mask_CM4X.ross_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_filchner_CM4x_historical=(mask_CM4X.filchner_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_peninsula_CM4x_historical= (mask_CM4X.peninsula_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_loweast_CM4x_historical= (mask_CM4X.loweast_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_lowwest_CM4x_historical= (mask_CM4X.lowwest_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])
weight_plateau_CM4x_historical= (mask_CM4X.plateau_CM4X*cm4x_his_speedmerge).weighted(weights_cm4x).mean(dim=['lat','lon'])

In [39]:
weight_ross_cm4x_annual_historical = ml.util.annual_average(weight_ross_CM4x_historical,tcoord='time')
weight_filchner_cm4x_annual_historical= ml.util.annual_average(weight_filchner_CM4x_historical, tcoord='time')
weight_peninsula_cm4x_annual_historical=ml.util.annual_average(weight_peninsula_CM4x_historical, tcoord='time')
weight_loweast_cm4x_annual_historical=ml.util.annual_average(weight_loweast_CM4x_historical, tcoord='time')
weight_lowwest_cm4x_annual_historical=ml.util.annual_average(weight_lowwest_CM4x_historical, tcoord='time')
weight_plateau_cm4x_annual_historical=ml.util.annual_average(weight_plateau_CM4x_historical, tcoord='time')

In [41]:
ross_total_std_CM4x= weight_ross_cm4x_annual_historical.std('time')
filchner_total_std_CM4x= weight_filchner_cm4x_annual_historical.std('time')
peninsula_total_std_CM4x=weight_peninsula_cm4x_annual_historical.std('time')
loweast_total_std_CM4x=weight_loweast_cm4x_annual_historical.std('time')
lowwest_total_std_CM4x=weight_lowwest_cm4x_annual_historical.std('time')
plateau_total_std_CM4x=weight_plateau_cm4x_annual_historical.std('time')

In [43]:
print(ross_total_std_CM4x.values)
print(filchner_total_std_CM4x.values)
print(peninsula_total_std_CM4x.values)
print(loweast_total_std_CM4x.values)
print(lowwest_total_std_CM4x.values)
print(plateau_total_std_CM4x.values)

0.08563159452774731
0.09880345534111222
0.11616729623324284
0.04899005280025134
0.06580244510510012
0.06702133318191811


In [45]:
ross_total_CM4X_his= weight_ross_CM4x_historical.mean('time')
filchner_total_CM4X_his=weight_filchner_CM4x_historical.mean('time')
peninsula_total_CM4X_his=weight_peninsula_CM4x_historical.mean('time')
loweast_total_CM4X_his=weight_loweast_CM4x_historical.mean('time')
lowwest_total_CM4X_his=weight_lowwest_CM4x_historical.mean('time')
plateau_total_CM4X_his=weight_plateau_CM4x_historical.mean('time')

In [46]:
ross_total_val_CM4X_his= print(ross_total_CM4X_his.values)
filchner_total_val_CM4X_his=print(filchner_total_CM4X_his.values)
peninsula_total_val_CM4X_his=print(peninsula_total_CM4X_his.values)
loweast_total_val_CM4X_his=print(loweast_total_CM4X_his.values)
lowwest_total_val_CM4X_his=print(lowwest_total_CM4X_his.values)
plateau_total_val_CM4X_his=print(plateau_total_CM4X_his.values)

2.7948578072203487
2.4545504671235236
3.046108256536738
5.913807171022635
4.066487088117113
4.661868222546543


# CM4x SSP58

In [48]:
cm4x_uas_ssp585=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/ssp585/atmos/monthly/u_ref/*.nc')
cm4x_uas_ssp585

<xarray.Dataset>
Dimensions:     (time: 1020, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 2015-01-16 12:00:00 ... 2099-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    u_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.201501-201912.u_ref.nc
    title:                 CM4_ssp585_c192_OM4p25
    associated_files:      area: 20190101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Thu Feb 16 13:17:56 2023
    hostname:              pp033
    history:               fregrid --standard_dimension --input_mosaic C192_m...

In [ ]:
.sel(time=slice('2081-01-16', '2100-12-16'))

In [49]:
cm4x_vas_ssp585=xr.open_mfdataset('/backup/tuq41374/model_output/cm4x/p25/ssp585/atmos/monthly/v_ref/*.nc')
cm4x_vas_ssp585

<xarray.Dataset>
Dimensions:     (time: 1020, bnds: 2, lat: 360, lon: 576)
Coordinates:
  * bnds        (bnds) float64 1.0 2.0
  * lat         (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon         (lon) float64 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time        (time) object 2015-01-16 12:00:00 ... 2099-12-16 12:00:00
Data variables:
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 360, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 576, 2), meta=np.ndarray>
    time_bnds   (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    v_ref       (time, lat, lon) float32 dask.array<chunksize=(60, 360, 576), meta=np.ndarray>
Attributes:
    filename:              atmos.201501-201912.v_ref.nc
    title:                 CM4_ssp585_c192_OM4p25
    associated_files:      area: 20190101.grid_spec.nc
    grid_type:             regular
    grid_tile:             N/A
    code_release_version:  19.1
    git_hash:              c97c1f6afab0df3545ee82c337d51accc4205bc3
    creationtime:          Thu Feb 16 13:17:56 2023
    hostname:              pp033
    history:               fregrid --standard_dimension --input_mosaic C192_m...

# ESM4 Historical 

# ESM4 SSP585

# CM4 Historical

# CM4 SSp585

# ERA5 